<a href="https://colab.research.google.com/github/ab17254/dissertation/blob/main/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza

     |████████████████████████████████| 286kB 29.2MB/s 


In [2]:
# Upload config.py with API keys
from google.colab import files
uploaded = files.upload()

Saving config.py to config.py


In [3]:
import os
from config import KaggleKey
os.environ['KAGGLE_USERNAME'] = KaggleKey.kaggle_username
os.environ['KAGGLE_KEY'] = KaggleKey.kaggle_key
!kaggle datasets download -d albrace/twitter-data

 98% 581M/592M [00:06<00:00, 121MB/s]
100% 592M/592M [00:07<00:00, 87.1MB/s]


In [4]:
!unzip /content/twitter-data.zip -d /content/data

Archive:  /content/twitter-data.zip
  inflating: /content/data/clean_twitter_data/all_users.csv  
  inflating: /content/data/clean_twitter_data/pol_actor_users.csv  
  inflating: /content/data/twitter_data/all_twitter_data.csv  
  inflating: /content/data/twitter_data/political_twitter_data.csv  


In [5]:
import string
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

import stanza
stanza.download('en')

sns.set_style('whitegrid')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


2021-06-08 12:16:37 INFO: Downloading default packages for language: en (English)...
2021-06-08 12:18:24 INFO: Finished downloading models and saved to /root/stanza_resources.


In [6]:
nlp = stanza.Pipeline('en', processors='tokenize,sentiment')

2021-06-08 12:18:36 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2021-06-08 12:18:36 INFO: Use device: cpu
2021-06-08 12:18:36 INFO: Loading: tokenize
2021-06-08 12:18:36 INFO: Loading: sentiment
2021-06-08 12:18:36 INFO: Done loading processors!


In [26]:
pol_df = pd.read_csv('/content/data/twitter_data/political_twitter_data.csv', encoding='cp1252')
all_df = pd.read_csv('/content/data/twitter_data/all_twitter_data.csv',encoding='cp1252')

In [8]:
print(all_df.shape)
all_df.isnull().sum()

(2311105, 21)


tweet_date                          0
tweet_content                       0
tweet_id                            0
tweet_likes                         0
tweet_replies                       0
tweet_retweets                      0
tweet_quotes                        0
user_username                       0
user_id                             0
user_followers                      0
user_friends                        0
user_statuses                       0
user_verified                       0
user_url                            0
tweet_url                           0
mentioned_users               1688666
quotedTweet_id                1924546
quotedTweet_content           1924546
quotedTweet_username          1924546
quotedTweet_userID            1924546
quotedTweet_mentionedUsers    2228417
dtype: int64

In [9]:
print(pol_df.shape)
pol_df.isnull().sum()

(21360, 21)


tweet_date                        0
tweet_content                     0
tweet_id                          0
tweet_likes                       0
tweet_replies                     0
tweet_retweets                    0
tweet_quotes                      0
user_username                     0
user_id                           0
user_followers                    0
user_friends                      0
user_statuses                     0
user_verified                     0
user_url                          0
tweet_url                         0
mentioned_users               13765
quotedTweet_id                18654
quotedTweet_content           18654
quotedTweet_username          18654
quotedTweet_userID            18654
quotedTweet_mentionedUsers    20251
dtype: int64

In [10]:
pol_df['tweet_date'] = pol_df['tweet_date'].map(lambda x: str(x)[:-15])
all_df['tweet_date'] = all_df['tweet_date'].map(lambda x: str(x)[:-15])

all_df['tweet_date'] = pd.to_datetime(all_df['tweet_date'])
all_df = all_df[~(all_df['tweet_date'] < '2017-04-18')]

pol_df['tweet_date'] = pd.to_datetime(pol_df['tweet_date'])

In [11]:
punc = list(string.punctuation)
stop = stopwords.words('english') + punc + ['rt', 'via']

tt = TweetTokenizer()

In [12]:
def hashtag_extract(s):
  hashtags = []
  for i in s:
    ht = re.findall(r"#(\w+)", i)
    hashtags.append(ht)
  return hashtags

In [13]:
def fix_encode(df):
  df['tweet_content'] = df['tweet_content'].str.replace('&amp', '&')
  df['tweet_content'] = df['tweet_content'].str.replace('b"', '')
  df['tweet_content'] = df['tweet_content'].str.replace("b'", '')
  return df

In [14]:
def join_punct(s):
    return ''.join(ch for ch in s if ch not in punc)

In [15]:
def remove_emoji(s):
    emoji_pattern = re.compile(
        r'(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff]|(?:\U0001f92d))',
        re.UNICODE)
    return emoji_pattern.sub(r'', s)

In [16]:
def remove_url(s):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    no_url = url_pattern.sub(r'', s)
    return no_url

In [17]:
def clean_tweet(s):
  clean_tweets = []
  for i in s:
    tweet_token = tt.tokenize(join_punct(remove_emoji(remove_url(i.lower()))))
    no_stop_words = [token for token in tweet_token if not token in stop]
    clean_tweets.append(no_stop_words)
  return clean_tweets

In [18]:
def preprocessing_user(df):

  df['mentioned_users']=df['mentioned_users'].fillna("")
  mentioned_user = []
  for i, user in enumerate(df['mentioned_users']):
    usernames = user.split("username='")
    username_list = []
    for username in usernames:
      username_list.append(re.findall("^(.+?)'", username))
    mentioned_user.append(username_list[1:])
  df = df.drop(columns=['mentioned_users'])
  df['mentioned_users'] = mentioned_user
  df['mentioned_users'] = df['mentioned_users'].apply(lambda x: np.nan if len(x)==0 else x)

  df['quotedTweet_mentionedUsers']=df['quotedTweet_mentionedUsers'].fillna("")
  mentioned_user = []
  for i, user in enumerate(df['quotedTweet_mentionedUsers']):
    usernames = user.split("username='")
    username_list = []
    for username in usernames:
      username_list.append(re.findall("^(.+?)'", username))
    mentioned_user.append(username_list[1:])
  df = df.drop(columns=['quotedTweet_mentionedUsers'])
  df['quotedTweet_mentionedUsers'] = mentioned_user
  df['quotedTweet_mentionedUsers'] = df['quotedTweet_mentionedUsers'].apply(lambda x: np.nan if len(x)==0 else x)

  return df

In [19]:
def clean_sentiment(s):
  clean_tweets = []
  for i in s:
    x = join_punct(remove_url(i.lower()))
    clean_tweets.append(x)
  return clean_tweets

In [29]:
def sentiment(df):
  rows = []
  for tweet in pol_df['clean_sentiment_content']:
    doc = nlp(tweet)
    for sentence in doc.sentences:
      row={
          'text': sentence.text,
          'sentiment': sentence.sentiment
      }
    rows.append(row)
  return pd.DataFrame(rows)

In [28]:
pol_df = fix_encode(pol_df)
pol_df['hashtags'] = hashtag_extract(pol_df['tweet_content'])
pol_df['clean_tweet_content'] = clean_tweet(pol_df['tweet_content'])
pol_df = preprocessing_user(pol_df)
pol_df['clean_sentiment_content'] = clean_sentiment(pol_df['tweet_content'])
sentiment_df = sentiment(pol_df)
pol_sent_df=sentiment_df.merge(right=pol_df, left_on='text', right_on='clean_sentiment_content')
pol_sent_df = pol_sent_df.drop(columns=['text', 'clean_sentiment_content'])

In [37]:
all_df = fix_encode(all_df)
all_df['hashtags'] = hashtag_extract(all_df['tweet_content'])
all_df['clean_tweet_content'] = clean_tweet(all_df['tweet_content'])
# all_df = preprocessing_user(all_df)
all_df['clean_sentiment_content'] = clean_sentiment(all_df['tweet_content'])
sentiment_df = sentiment(all_df)
all_sent_df=sentiment_df.merge(right=all_df, left_on='text', right_on='clean_sentiment_content')
all_sent_df = all_sent_df.drop(columns=['text', 'clean_sentiment_content'])

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
g = sns.histplot(data=pol_df, x='tweet_date')
plt.setp(g.get_xticklabels(), rotation=45)
plt.show()

In [40]:
!mkdir output

In [41]:
pol_sent_df.to_csv('output/pol_actor_users.csv')
all_sent_df.to_csv('output/all_users.csv')

In [42]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/all_users.csv (deflated 83%)
  adding: content/output/pol_actor_users.csv (deflated 87%)
